In [1]:
import numpy as np
import cvxpy as cp

In [2]:
# Example data (expected returns and covariance matrix)
# Thay thế bằng dữ liệu lịch sử các cổ phiếu
expected_returns = np.array([0.1, 0.08, 0.12, 0.09, 0.11])
covariance_matrix = np.array([
    [0.1, 0.03, 0.02, 0.01, 0.05],
    [0.03, 0.12, 0.01, 0.02, 0.04],
    [0.02, 0.01, 0.15, 0.03, 0.03],
    [0.01, 0.02, 0.03, 0.09, 0.02],
    [0.05, 0.04, 0.03, 0.02, 0.1]
])

# Number of assets
n_assets = len(expected_returns)

# Define the variables
weights = cp.Variable(n_assets)

# Define the objective function (minimize portfolio risk)
risk = cp.quad_form(weights, covariance_matrix)
objective = cp.Minimize(risk)

# Define the constraints (sum of weights = 1, weights >= 0)
constraints = [cp.sum(weights) == 1, weights >= 0]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Print the results
print("Optimal Portfolio Weights:")
for i in range(n_assets):
    print(f"Asset {i+1}: {weights.value[i]:.4f}")

# Calculate and print portfolio expected return and risk
portfolio_return = expected_returns.dot(weights.value)
portfolio_risk = np.sqrt(weights.value.T @ covariance_matrix @ weights.value)
print("\nPortfolio Metrics:")
print(f"Expected Return: {portfolio_return:.4f}")
print(f"Portfolio Risk: {portfolio_risk:.4f}")


Optimal Portfolio Weights:
Asset 1: 0.2419
Asset 2: 0.1813
Asset 3: 0.1450
Asset 4: 0.3190
Asset 5: 0.1128

Portfolio Metrics:
Expected Return: 0.0972
Portfolio Risk: 0.2034


In [3]:
import numpy as np
import cvxpy as cp

# Example data (expected returns and covariance matrix)
expected_returns = np.array([0.1, 0.08, 0.12, 0.09, 0.11])
covariance_matrix = np.array([
    [0.1, 0.03, 0.02, 0.01, 0.05],
    [0.03, 0.12, 0.01, 0.02, 0.04],
    [0.02, 0.01, 0.15, 0.03, 0.03],
    [0.01, 0.02, 0.03, 0.09, 0.02],
    [0.05, 0.04, 0.03, 0.02, 0.1]
])

# Number of assets
n_assets = len(expected_returns)

# Define the variables
weights = cp.Variable(n_assets)

# Define the objective function (maximize portfolio return)
objective = cp.Maximize(expected_returns @ weights)

# Define the constraints (sum of weights = 1, weights >= 0)
constraints = [cp.sum(weights) == 1, weights >= 0]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Print the results
print("Optimal Portfolio Weights:")
for i in range(n_assets):
    print(f"Asset {i+1}: {weights.value[i]:.4f}")

# Calculate and print portfolio expected return and risk
portfolio_return = expected_returns.dot(weights.value)
portfolio_risk = np.sqrt(weights.value.T @ covariance_matrix @ weights.value)
print("\nPortfolio Metrics:")
print(f"Expected Return: {portfolio_return:.4f}")
print(f"Portfolio Risk: {portfolio_risk:.4f}")


Optimal Portfolio Weights:
Asset 1: 0.0000
Asset 2: 0.0000
Asset 3: 1.0000
Asset 4: 0.0000
Asset 5: 0.0000

Portfolio Metrics:
Expected Return: 0.1200
Portfolio Risk: 0.3873


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
